In [38]:
import numpy as np
import pandas as pd
import openpyxl
from conditionalAssetPricingLogMarginalLikelihoodTauClass import Model, printFactorsAndPredictorsProbabilities
import os

# === Step 1: Setup ===
homeDir = os.path.dirname(os.getcwd())
dataDir = os.path.join(os.path.join(homeDir,'Complemetary Code Files for Submission'), 'Data')

directory_name_prefix = homeDir + 'conditionalTauDMN0F14M13s'

# === Step 2: Choose number of predictors to use ===
n_predictors_to_use = 2  # 🔧 Set this to anything between 1 and 13
assert 1 <= n_predictors_to_use <= 13, "You must choose between 1 and 13 predictors"

# === Step 3: Load test assets (returns) ===
returns = pd.read_excel(os.path.join(dataDir,'Returns.xlsx'), sheet_name="Industry_returns")
returns = returns.iloc[:-1]  # remove first row

returns = returns.drop(columns=['Grand Total'])  # remove Date column
#for col in returns.columns:
#    returns[col] = returns[col] * 100
# === Step 4: Load and clean factors ===
factors = pd.read_csv(os.path.join(dataDir,'factors-20.csv'))
factors = factors.drop(columns=['MKTRF','SMB*','MKT','CON','IA', 'ROE', 'ME'])  # remove redundant/duplicated
factorsNames = factors.columns.drop('Date')
for col in factors.columns:
   if col != 'Date':
        factors[col] = factors[col]



In [39]:
print(returns.columns.tolist())

['Row Labels', 'Agriculture', 'Construction', 'Finance', 'Manufacturing', 'Mining', 'Retail', 'Services', 'Transportation', 'Utilities', 'Wholesale']


In [40]:
returns.drop(columns="Row Labels", inplace=True)
returns

,Agriculture,Construction,Finance,Manufacturing,Mining,Retail,Services,Transportation,Utilities,Wholesale
0,0.057788,0.026084,-0.005252,0.003019,0.009399,0.010534,0.029401,0.023169,-0.001782,1.272246e-02
1,-0.057842,-0.079316,-0.026690,-0.039546,-0.042441,-0.046026,-0.041265,-0.014670,-0.036624,-4.880501e-02
2,-0.005860,0.024880,0.013199,0.004420,-0.012095,0.015887,0.019635,0.012285,0.000911,2.242321e-02
3,0.074653,0.050209,0.078998,0.067757,0.021494,0.080548,0.105231,0.072612,0.021078,8.577112e-02
4,-0.025000,0.110476,0.094256,0.089970,0.053907,0.086127,0.076630,0.088947,0.003256,9.935179e-02
...,...,...,...,...,...,...,...,...,...,...
344,0.187080,0.026953,0.016139,0.029970,-0.025261,0.031498,0.031392,0.022141,0.021117,-8.323987e+06
345,0.000989,0.015533,0.029042,0.015371,-0.086312,0.067173,0.012764,0.009039,-0.019266,3.422338e-02
346,0.000247,0.069760,0.030724,0.054848,0.077258,0.054438,0.042749,0.065477,0.058642,6.138147e-02
347,0.005184,0.072309,0.020243,0.036036,0.085918,0.021556,0.034480,0.025617,0.037270,2.387625e-02


In [41]:

# === Step 5: Load and clean predictors ===
predictors = pd.read_csv(os.path.join(dataDir,'Z - 197706.csv'))
if 'Unnamed: 0' in predictors.columns:
    predictors = predictors.drop(columns='Unnamed: 0')

# Define full predictor index list
full_predictor_list = predictors.columns.drop('Date')
predictors[full_predictor_list] = predictors[full_predictor_list] / 100
significantPredictors = np.arange(n_predictors_to_use)
predictorsNames = full_predictor_list[significantPredictors]

# Subset actual predictor data
predictors = predictors[['Date'] + predictorsNames.tolist()]





In [42]:
print(predictors.describe().T[['mean', 'std']])

        mean       std
dp -0.036696  0.004397
dy -0.036629  0.004407


In [43]:
factors

,Date,Mkt-RF,SMB,HML,RMW,CMA,MMOM,PEAD,FIN,QMJ,BAB,MGMT,PERF,LIQ,IFCR
0,197706,0.0471,0.0207,-0.0064,0.0088,-0.0122,0.000025,0.002542,-0.034968,-0.008333,-0.002229,-0.009977,0.010016,0.0204,0.067700
1,197707,-0.0169,0.0182,-0.0059,0.0083,0.0001,0.003100,-0.007776,-0.018042,0.009184,0.028603,0.011125,0.003183,0.0065,0.007800
2,197708,-0.0175,0.0089,-0.0279,0.0096,-0.0062,-0.017700,0.019276,-0.004259,0.030540,0.003437,-0.006110,0.026611,0.0295,-0.017700
3,197709,-0.0027,0.0156,-0.0049,0.0123,-0.0084,0.020500,0.010915,-0.007109,0.003504,0.004118,-0.001179,0.017279,0.0136,-0.022500
4,197710,-0.0438,0.0150,0.0172,-0.0030,-0.0046,-0.001200,-0.008172,0.016989,0.010512,0.001830,0.008145,0.003757,-0.0069,-0.068400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,201608,0.0050,0.0172,0.0334,-0.0133,-0.0039,-0.031300,-0.010773,-0.007482,0.001626,-0.018531,0.018547,-0.049411,0.0076,0.077112
471,201609,0.0025,0.0174,-0.0149,-0.0229,-0.0009,-0.005500,0.039229,-0.030924,-0.031290,-0.001611,-0.009430,-0.021728,0.0235,-0.032019
472,201610,-0.0202,-0.0401,0.0416,0.0119,0.0024,0.006400,-0.013634,0.042513,0.027660,0.011112,0.032443,0.020009,-0.0101,0.062624
473,201611,0.0486,0.0681,0.0829,-0.0010,0.0370,-0.042100,0.023165,0.016926,0.006325,-0.015986,0.048981,-0.076985,0.0227,0.134509


In [44]:

# === Step 6: Truncate all to shared sample size and index ===
start_date = '1997-01-01'
end_date = '2006-12-01'
date_index = pd.date_range(start=start_date, end=end_date, freq='MS')
n = len(date_index)

returns = returns.iloc[:n].copy()

#returns_sim = returns_sim.iloc[:n].copy()

factors = factors.iloc[:n].copy()
predictors = predictors.iloc[:n].copy()
# === Convert decimal returns to percent returns ===

returns.index = date_index
factors.index = date_index
predictors.index = date_index

# === Step 7: Reset index and rename for model compatibility ===
returns_reset = returns.reset_index().rename(columns={'index': 'Date'})
#returns_reset.drop(columns='Row Labels', inplace=True) 

#returns_sim_reset = returns_sim.reset_index().rename(columns={'index': 'Date'})

factors_reset = factors.reset_index().rename(columns={'index': 'Date'})
predictors_reset = predictors.reset_index().rename(columns={'index': 'Date'})


In [45]:

returns_reset

,Date,Agriculture,Construction,Finance,Manufacturing,Mining,Retail,Services,Transportation,Utilities,Wholesale
0,1997-01-01,0.057788,0.026084,-0.005252,0.003019,0.009399,0.010534,0.029401,0.023169,-0.001782,0.012722
1,1997-02-01,-0.057842,-0.079316,-0.026690,-0.039546,-0.042441,-0.046026,-0.041265,-0.014670,-0.036624,-0.048805
2,1997-03-01,-0.005860,0.024880,0.013199,0.004420,-0.012095,0.015887,0.019635,0.012285,0.000911,0.022423
3,1997-04-01,0.074653,0.050209,0.078998,0.067757,0.021494,0.080548,0.105231,0.072612,0.021078,0.085771
4,1997-05-01,-0.025000,0.110476,0.094256,0.089970,0.053907,0.086127,0.076630,0.088947,0.003256,0.099352
...,...,...,...,...,...,...,...,...,...,...,...
115,2006-08-01,-0.012346,0.026243,0.017702,0.066206,0.151908,0.053828,0.028855,0.066102,-0.006920,0.062613
116,2006-09-01,0.064979,0.005359,0.039357,0.031045,-0.031330,0.023229,0.047295,0.015320,0.031129,0.028927
117,2006-10-01,-0.059821,0.001530,-0.045373,-0.005444,-0.010035,-0.064868,0.006578,-0.006828,-0.029709,-0.027107
118,2006-11-01,-0.202278,-0.352102,-0.263182,-0.297122,-0.311007,-0.304169,-0.295074,-0.261020,-0.096674,-0.297347


In [46]:
returns

,Agriculture,Construction,Finance,Manufacturing,Mining,Retail,Services,Transportation,Utilities,Wholesale
1997-01-01,0.057788,0.026084,-0.005252,0.003019,0.009399,0.010534,0.029401,0.023169,-0.001782,0.012722
1997-02-01,-0.057842,-0.079316,-0.026690,-0.039546,-0.042441,-0.046026,-0.041265,-0.014670,-0.036624,-0.048805
1997-03-01,-0.005860,0.024880,0.013199,0.004420,-0.012095,0.015887,0.019635,0.012285,0.000911,0.022423
1997-04-01,0.074653,0.050209,0.078998,0.067757,0.021494,0.080548,0.105231,0.072612,0.021078,0.085771
1997-05-01,-0.025000,0.110476,0.094256,0.089970,0.053907,0.086127,0.076630,0.088947,0.003256,0.099352
...,...,...,...,...,...,...,...,...,...,...
2006-08-01,-0.012346,0.026243,0.017702,0.066206,0.151908,0.053828,0.028855,0.066102,-0.006920,0.062613
2006-09-01,0.064979,0.005359,0.039357,0.031045,-0.031330,0.023229,0.047295,0.015320,0.031129,0.028927
2006-10-01,-0.059821,0.001530,-0.045373,-0.005444,-0.010035,-0.064868,0.006578,-0.006828,-0.029709,-0.027107
2006-11-01,-0.202278,-0.352102,-0.263182,-0.297122,-0.311007,-0.304169,-0.295074,-0.261020,-0.096674,-0.297347


In [47]:
# === Step 8: Define model parameters ===
Tau = 1.5
index_end_of_estimation = 118  # Half of length of returns

# === Step 9: Instantiate model ===
model = Model(rr=returns_reset,
              ff=factors_reset,
              zz=predictors_reset,
              significantPredictors=significantPredictors,
              Tau=Tau,
              indexEndOfEstimation=index_end_of_estimation,
              key_demean_predictors=True)


Tau= 1.500000 
Date    2006-11-01 00:00:00
Date                 198704
Name: 118, dtype: object
Date    2006-11-01 00:00:00
Date               4/1/1987
Name: 118, dtype: object
Date              2006-11-01 00:00:00
Agriculture                 -0.202278
Construction                -0.352102
Finance                     -0.263182
Manufacturing               -0.297122
Mining                      -0.311007
Retail                      -0.304169
Services                    -0.295074
Transportation               -0.26102
Utilities                   -0.096674
Wholesale                   -0.297347
Name: 118, dtype: object
Market SR^2 in the estimation period= 0.023557
ZEstimation mean and std
[-0.03075668 -0.03066261]
[0.00165519 0.00160418]
ZTest mean and std
[-0.03535978 -0.03547494]
[0. 0.]
After demeaning
ZEstimation mean and std
[3.40970190e-15 1.81022805e-15]
[1. 1.]
ZTest mean and std
[-2.78099991 -2.99986675]
[0. 0.]
REstimation.shape= (118, 10)
RTest.shape= (1, 10)
FEstimation.shape= (1

In [ ]:
#Calculate marginal likelihood
CLMLList = []
(CLMLU, factorsNames, factorsProbabilityU, predictorsNames, predictorsProbabilityU, \
                    T0IncreasedFraction, T0Max, T0Min, T0Avg, T0_div_T0_plus_TAvg, T_div_T0_plus_TAvg, \
                    CLMLR, factorsProbabilityR, predictorsProbabilityR) = \
                        model.conditionalAssetPricingLogMarginalLikelihoodTauNumba()

CLMLCombined = np.concatenate((CLMLU, CLMLR), axis=0)
CMLCombined  = np.exp(CLMLCombined - max(CLMLCombined)); CMLCombined = CMLCombined / np.sum(CMLCombined)
print('probability of mispricing    = %f' %(np.sum(CMLCombined[0: len(CLMLU)])))
print('probability of no mispricing = %f' %(np.sum(CMLCombined[len(CLMLU):  ])))
factorsProbability = np.sum(CMLCombined[0:len(CLMLU)])*factorsProbabilityU + np.sum(CMLCombined[len(CLMLU): ]) *factorsProbabilityR

predictorsProbability = np.sum(CMLCombined[0:len(CLMLU)])*predictorsProbabilityU + np.sum(CMLCombined[len(CLMLU): ]) *predictorsProbabilityR

CLMLList.append({"Tau": Tau,     \
                            "LMLU" : CLMLU, "LMLR" :CLMLR,  \
                            "factorsProbability"    : factorsProbability,    \
                            "predictorsProbability" : predictorsProbability, \
                            "T0IncreasedFraction" : T0IncreasedFraction,     \
                            "T0Max" : T0Max, "T0Min" : T0Min, "T0Avg" : T0Avg,   \
                            "T0divT0plusTAvg" : T0_div_T0_plus_TAvg, "TdivT0plusTAvg" : T_div_T0_plus_TAvg, \
                            "MisprisingProb" : np.sum(CMLCombined[0: len(CLMLU)])})

***** conditionalAssetPricingLogMarginalLikelihoodTauNumba **** 


In [ ]:
CLMLU = CLMLList[0]["LMLU"]
CLMLR = CLMLList[0]["LMLR"]
CLMLCombined = np.concatenate((CLMLU, CLMLR), axis=0)
CMLCombined  = np.exp(CLMLCombined - max(CLMLCombined)); CMLCombined = CMLCombined / np.sum(CMLCombined)
print('probability of mispricing    = %f' %(np.sum(CMLCombined[0: len(CLMLU)])))
print('probability of no mispricing = %f' %(np.sum(CMLCombined[len(CLMLU):  ])))
factorsProbability = CLMLList[0]["factorsProbability"]
predictorsProbability = CLMLList[0]["predictorsProbability"]

printFactorsAndPredictorsProbabilities(model.factorsNames, factorsProbability, model.predictorsNames,
                                                   predictorsProbability)

probability of mispricing    = 0.459515
probability of no mispricing = 0.540485
Probabilities of factors
Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'MMOM', 'PEAD', 'FIN', 'QMJ',
       'BAB', 'MGMT', 'PERF', 'LIQ', 'IFCR'],
      dtype='object')
[1.         0.68999823 0.57767329 0.58172496 0.41891905 0.56701587
 0.4868556  0.52603499 0.53975936 0.85123272 0.72093479 0.95232846
 0.39854677 0.9666901 ]
Sorted Probabilities of factors
Index(['Mkt-RF', 'IFCR', 'PERF', 'BAB', 'MGMT', 'SMB', 'RMW', 'HML', 'MMOM',
       'QMJ', 'FIN', 'PEAD', 'CMA', 'LIQ'],
      dtype='object')
[1.         0.9666901  0.95232846 0.85123272 0.72093479 0.68999823
 0.58172496 0.57767329 0.56701587 0.53975936 0.52603499 0.4868556
 0.41891905 0.39854677]
Probabilities of Predictors
Index(['dp', 'dy'], dtype='object')
[0.96979639 0.96577656]
Sorted Probabilities of Predictors
Index(['dp', 'dy'], dtype='object')
[0.96979639 0.96577656]


In [ ]:
nModelsInPrediction = 1000
(returns_OOS , returns_square_OOS, returns_interactions_OOS, \
                        covariance_matrix_OOS, covariance_matrix_no_ER_OOS, \
                        returns_IN, returns_square_IN, returns_interactions_IN, \
                        covariance_matrix_IN, covariance_matrix_no_ER_IN, \
                        cumulative_probability) = \
                        model.conditionalAssetPricingOOSTauNumba(CMLCombined,nModelsInPrediction)

***** conditionalAssetPricingOOSTauNumba **** 
Sum of probabilities= 1.000000
Number of models to use in prediction= 1000 
Cumulative probabilities the 555 models in ModelsIndices= 0.629118
Model  0 out of 555


ValueError: operands could not be broadcast together with shapes (118,24) (118,14) (118,24) 

In [ ]:
returns_OOS

array([[0.0046179 , 0.00132796, 0.00298019, 0.00252853, 0.00261804,
        0.00583611, 0.0042713 , 0.00800194, 0.00343248, 0.01016643,
        0.00606284, 0.00532563, 0.00384799, 0.00844048]])